<a href="https://colab.research.google.com/github/jh941213/DeepLearing/blob/main/CNNMNIST_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
import torch.nn as nn


In [46]:
#텐서란 2차원의 객체이다. 벡터는 1차원 텐서 / 객체는 2차원 텐서 
inputs = torch.Tensor(1,1,28,28) # 임의의 텐서를 만듭니다 
print("텐서의 크기: {}".format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [47]:
# 2 합성곱층과 풀링 선언하기 
#Conv2d = 컨볼루션 신경망 (필터수, 행, 열, padding(경계처리방법))
#1채널을 받아서 32채널을 뽑아냄
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [48]:
# 합성곱층 32채널을 받아 64채널을 뽑아냄
conv2 = nn.Conv2d(32,64, 3, padding=(1,1))
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [49]:
#맥스풀링 구현
pool = nn.MaxPool2d(2) # 2를 넣으면 커널size와 stride가 2로 설정됨
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [50]:
out = conv1(inputs)# 임의의텐서를 conv1
print(out.shape)

torch.Size([1, 32, 28, 28])


In [51]:
out = pool(out) #maxpool링을 통과시켜
print(out.shape) #padding 1을 주면  3x3 커널을 사용하면 크기가 보존 된다
#32채널의 28, 28 텐서가 되었다

torch.Size([1, 32, 14, 14])


In [52]:
out = conv2(out) #conv2 채널이 64이기 때문이다
print(out.shape)
#64채널의 14, 14 텐서가 되었다.

torch.Size([1, 64, 14, 14])


In [53]:
out = pool(out) 
print(out.shape) #이 텐서의 크기는 1 x 64 x 7 x 7

torch.Size([1, 64, 7, 7])


In [54]:
out.size(0) # out의 첫번째 차원은 1이다. 
out.size(1) # out의 두번째 차원은 64이다
out.size(2) # out의 세번째 차원은 7이다.
out.size(3) # out의 네번째 차원은 7이다


7

In [55]:
#첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라!?
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [56]:
fc = nn.Linear(3136, 10) # 전결합층 통과 출력쳥으로 10개의 뉴런을 배치하여 10개 차원의 텐서변환
out = fc(out)


In [57]:
print(out.shape)

torch.Size([1, 10])


In [59]:
#CNN 으로 mnist 분류하기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init


In [64]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
#만약 GPU를 사용 가능하다면 device 값이 cuda가 되고 , 아니라면 cpu 가 됩니다.

#랜덤 시드 고정
torch.manual_seed(777)
#GPU 사용 가능일 경우 랜덤 시드 고정
if device =='cuda':
    torch.cuda_manual_seed_all(777)

In [63]:
#쓸 파라미터 설정
learning_rate = 0.001
training_epochs = 15 #전체 데이터를 15번 학습
batch_size = 100


In [66]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [67]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True
                                          )

In [69]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
            # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
    # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [71]:
# CNN 모델 정의
model = CNN().to(device)

In [72]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [73]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [74]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225628644
[Epoch:    2] cost = 0.0630425066
[Epoch:    3] cost = 0.046389699
[Epoch:    4] cost = 0.0374845006
[Epoch:    5] cost = 0.031433057
[Epoch:    6] cost = 0.0261363648
[Epoch:    7] cost = 0.022024285
[Epoch:    8] cost = 0.0183657762
[Epoch:    9] cost = 0.0162043925
[Epoch:   10] cost = 0.0134079289
[Epoch:   11] cost = 0.0102126868
[Epoch:   12] cost = 0.00976756122
[Epoch:   13] cost = 0.00878930092
[Epoch:   14] cost = 0.00629561162
[Epoch:   15] cost = 0.00555317383


In [75]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9873999953269958
